# 

https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html

In [108]:
from __future__ import print_function
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range
import math
def euDis(c1,c2):
    return math.sqrt((c1[0]-c2[0])**2+(c1[1]-c2[1])**2)

In [48]:
HER_GRID = 20
VER_GRID = 20
HER_LEN = 1800
VER_LEN = 1200
a_col = 8
a_row = 5
hopDis = 3
COL = HER_LEN//HER_GRID
ROW = VER_LEN//VER_GRID
def grid_anchors(rowCells,colCells,row=5,col=8):
    st_w=colCells//(col+1)
    st_h=rowCells//(row+1)
    #print(st_w,st_h)
    ini_w=0
    ini_h=0
    anchors = []
    for i in range(row):
        ini_h+=st_h
        ini_w=0
        for j in range(col):
            ini_w+=st_w
            anchors.append([ini_w,ini_h])
    return anchors

anchors = grid_anchors(ROW,COL)
PRElOCATION=50
hoplimits = 20

In [77]:
class GridPoint():
    def __init__(self, pStr):
        pStrList = pStr.split(" ")
        self.point = [int(x) for x in pStrList]
        self.x = self.point[0]
        self.y = self.point[1]
        
def encodeGPList(gpList, nRows):
    x = np.zeros((nRows, COL+ROW))
    #print(len(gpList))
    for i, p in enumerate(gpList):
        x[i, p.x] = 1
        yy = COL+p.y
        if(yy>=COL+ROW):
            yy=COL+ROW-1
        x[i, yy] = 1
    return x
                
class constraint():
    def __init__(self,consStr,gpList):
        self.gpList = gpList
        self.hopInfo=[]
        if consStr == "#": 
            self.hopInfo = [1,0,0,0,0,0,0]
        elif consStr == "*": 
            self.hopInfo = [0,1,0,0,0,0,0]
        else:
            cStrList = consStr.replace("(","").replace(")","").split(";")[:5]
            self.hopInfo = [0,0]+[int(float(x)) for x in cStrList]

    def encodeConstraints(self,anchors,hopLimits):
        col = max(len(anchors),hopLimits)
        x = np.zeros((5, col))
        for i, p in enumerate(self.hopInfo[2:]):
            x[i, p] = 1
        
        return x
        
    def findFirstLastAppear(self):
        leftAnchor = anchors[self.hopInfo[2]]
        rightAnchor = anchors[self.hopInfo[3]]
        circleCenter = anchors[self.hopInfo[4]]
        x=circleCenter[0]
        y=circleCenter[1]
        r=self.hopInfo[6]*hopDis
        a=self.hopInfo[5]*hopDis
        a2=a-hopDis
        
        h1x=leftAnchor[0]
        h1y=leftAnchor[1]
        h2x=rightAnchor[0]
        h2y=rightAnchor[1]

        rr=r*r 
        res = [-1,-1]
        for k,tup in enumerate(self.gpList):
            i=tup.x
            j=tup.y
            di=x-i
            dj=y-j
            if  (di*di+dj*dj<=rr and euDis((i,j),(h1x,h1y))-euDis((i,j),(h2x,h2y))<=2*a and euDis((i,j),(h1x,h1y))-euDis((i,j),(h2x,h2y))>=2*a2):
                res[0] = k
                break
        else: res[0] = k
        for k,tup in enumerate(self.gpList[::-1]):
            i=tup.x
            j=tup.y
            di=x-i
            dj=y-j
            if  (di*di+dj*dj<=rr and euDis((i,j),(h1x,h1y))-euDis((i,j),(h2x,h2y))<=2*a and euDis((i,j),(h1x,h1y))-euDis((i,j),(h2x,h2y))>=2*a2):
                res[1] = len(self.gpList)  - k 
                break
        else: res[1] = k 
        #print(res)
        return res
    
    def myEncoderInput(self,nRows):
        first,last = self.findFirstLastAppear()
        gpList = []
        
        if last-first>=nRows:
            gpList=self.gpList[first:first+nRows]
        else:
            if last-nRows>=0:
                gpList=self.gpList[last-nRows:last]
            else:
                gpList = self.gpList[:min(len(self.gpList),nRows)]
                
        self.encoderInput = encodeGPList(gpList,nRows)
        return self.encoderInput
        
    def decodeConstraint (self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
            
        return [x for x in x]

In [50]:
f = open("./dataExpect/50/preTrain_22", "r")
for t in f:
    print(t)
f.close()

117

62 40

61 40

62 39

61 39

61 38

60 39

60 38

59 39

59 38

58 38

59 37

58 37

57 37

58 36

57 36

56 37

56 36

55 36

56 35

55 35

55 34

54 35

54 34

53 35

53 34

52 34

53 33

52 33

51 33

52 32

51 32

51 31

50 32

50 31

49 32

49 31

48 31

49 30

48 30

47 30

48 29

47 29

46 29

45 29

46 28

45 28

45 27

44 28

44 27

43 28

43 27

42 27

43 26

42 26

42 25

41 26

41 25

40 26

40 25

39 25

40 24

38 25

39 24

40 23

38 24

39 23

38 23

38 22

37 23

37 22

36 23

36 22

35 22

34 22

35 21

34 21

33 21

32 21

31 21

32 22

31 22

31 23

31 24

32 23

31 25

32 24

31 26

32 25

32 26

32 27

32 28

32 29

32 30

31 29

32 31

31 30

31 31

30 30

31 32

30 31

30 32

30 33

29 32

30 34

29 33

29 34

29 35

29 36

28 36

29 37

28 37

28 38

28 39

27 38

28 40

27 39

27 40

(15; 36; 6; 3.000000; 10.000000; 3.000000; 3.000000)

(7; 20; 24; 6.000000; 9.000000; 3.000000; 3.000000)

(25; 11; 33; 4.000000; 11.000000; 3.000000; 3.000000)

(33; 4; 34; 3.

In [59]:
f = open("./dataExpect/50/preTrain_22", "r")
TAS_SIZE = int(f.readline())
gpList = []
for i in range(TAS_SIZE):
    gpList.append(GridPoint(f.readline()))
print([p.point for p in gpList])
firstConstraint = constraint(f.readline(),gpList)
f.close()

print(firstConstraint.hopInfo)
decoderInput = firstConstraint.encodeConstraints(anchors,hoplimits)
print(decoderInput)





[[62, 40], [61, 40], [62, 39], [61, 39], [61, 38], [60, 39], [60, 38], [59, 39], [59, 38], [58, 38], [59, 37], [58, 37], [57, 37], [58, 36], [57, 36], [56, 37], [56, 36], [55, 36], [56, 35], [55, 35], [55, 34], [54, 35], [54, 34], [53, 35], [53, 34], [52, 34], [53, 33], [52, 33], [51, 33], [52, 32], [51, 32], [51, 31], [50, 32], [50, 31], [49, 32], [49, 31], [48, 31], [49, 30], [48, 30], [47, 30], [48, 29], [47, 29], [46, 29], [45, 29], [46, 28], [45, 28], [45, 27], [44, 28], [44, 27], [43, 28], [43, 27], [42, 27], [43, 26], [42, 26], [42, 25], [41, 26], [41, 25], [40, 26], [40, 25], [39, 25], [40, 24], [38, 25], [39, 24], [40, 23], [38, 24], [39, 23], [38, 23], [38, 22], [37, 23], [37, 22], [36, 23], [36, 22], [35, 22], [34, 22], [35, 21], [34, 21], [33, 21], [32, 21], [31, 21], [32, 22], [31, 22], [31, 23], [31, 24], [32, 23], [31, 25], [32, 24], [31, 26], [32, 25], [32, 26], [32, 27], [32, 28], [32, 29], [32, 30], [31, 29], [32, 31], [31, 30], [31, 31], [30, 30], [31, 32], [30, 31],

In [60]:

encoderInput = firstConstraint.myEncoderInput(PRElOCATION)
print(encoderInput)
print(encoderInput[0])







50
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]


In [54]:
import os.path
from os import path

strokeRange = [20,194]
totalNumOfTestCases = 0
for sr in range(strokeRange[0],strokeRange[1]):
    for item in range(1,100000):
        fileName = "./dataExpect/"+str(sr)+"/preTrain_"+str(item)
        if path.exists(fileName):
            totalNumOfTestCases+=1
print (totalNumOfTestCases)

1417334


In [78]:
print('Vectorization...')


x = np.zeros((totalNumOfTestCases, PRElOCATION, ROW+COL), dtype=np.bool)
y = np.zeros((totalNumOfTestCases, 5, max(a_col*a_row,hoplimits)), dtype=np.bool)

ttc=0
for sr in range(strokeRange[0],strokeRange[1]):
    for item in range(1,100000):
        fileName = "./dataExpect/"+str(sr)+"/preTrain_"+str(item)
        if path.exists(fileName):
            f = open(fileName, "r")
            TAS_SIZE = int(f.readline())
            gpList = []
            for i in range(TAS_SIZE):
                gpList.append(GridPoint(f.readline()))
            firstConstraint = constraint(f.readline(),gpList)
            f.close()
            y[ttc] = firstConstraint.encodeConstraints(anchors,hoplimits)
            x[ttc] = firstConstraint.myEncoderInput(PRElOCATION)
            ttc+=1

Vectorization...


In [79]:
# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

In [80]:
# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

In [105]:
print('Training Data:')
print(x_train.shape)
print(y_train.shape)


Training Data:
(1275601, 50, 150)
(1275601, 5, 40)


In [106]:
print('Validation Data:')
print(x_val.shape)
print(y_val.shape)


Validation Data:
(141733, 50, 150)
(141733, 5, 40)


In [104]:
# Try replacing GRU, or SimpleRNN.
RNN = layers.LSTM
HIDDEN_SIZE = 256
BATCH_SIZE = 128
LAYERS = 1

In [127]:
from tensorflow.python.client import device_lib
import tensorflow as tf
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7885889664343187364
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 4372143343324139170
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 16320562271449302993
physical_device_desc: "device: XLA_GPU device"
]


In [128]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense

print('Build model...')
model = Sequential()
# "Encode" the input sequence using an RNN, producing an output of HIDDEN_SIZE.
# Note: In a situation where your input sequences have a variable length,
# use input_shape=(None, num_feature).
model.add(RNN(HIDDEN_SIZE, input_shape=(PRElOCATION, ROW+COL)))
# As the decoder RNN's input, repeatedly provide with the last output of
# RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
# length of output, e.g., when DIGITS=3, max output is 999+999=1998.

model.add(layers.RepeatVector(5))
# The decoder RNN could be multiple layers stacked or a single layer.
for _ in range(LAYERS):
    # By setting return_sequences to True, return not only the last output but
    # all the outputs so far in the form of (num_samples, timesteps,
    # output_dim). This is necessary as TimeDistributed in the below expects
    # the first dimension to be the timesteps.
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.
model.add(layers.TimeDistributed(layers.Dense(40, activation='softmax')))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

'''

encoder_inputs = Input(shape=(None, 50))
encoder = LSTM(256, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, 5))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the 
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(5, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)


'''




Build model...
Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_38 (LSTM)               (None, 256)               416768    
_________________________________________________________________
repeat_vector_9 (RepeatVecto (None, 5, 256)            0         
_________________________________________________________________
lstm_39 (LSTM)               (None, 5, 256)            525312    
_________________________________________________________________
time_distributed_11 (TimeDis (None, 5, 40)             10280     
Total params: 952,360
Trainable params: 952,360
Non-trainable params: 0
_________________________________________________________________


"\n\nencoder_inputs = Input(shape=(None, 50))\nencoder = LSTM(256, return_state=True)\nencoder_outputs, state_h, state_c = encoder(encoder_inputs)\n# We discard `encoder_outputs` and only keep the states.\nencoder_states = [state_h, state_c]\n\n# Set up the decoder, using `encoder_states` as initial state.\ndecoder_inputs = Input(shape=(None, 5))\n# We set up our decoder to return full output sequences,\n# and to return internal states as well. We don't use the \n# return states in the training model, but we will use them in inference.\ndecoder_lstm = LSTM(5, return_sequences=True, return_state=True)\ndecoder_outputs, _, _ = decoder_lstm(decoder_inputs,\n                                     initial_state=encoder_states)\ndecoder_dense = Dense(num_decoder_tokens, activation='softmax')\ndecoder_outputs = decoder_dense(decoder_outputs)\n\n# Define the model that will turn\n# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`\nmodel = Model([encoder_inputs, decoder_inpu

In [ ]:

def ga(rowCells,colCells,row=5,col=8):
    st_w=colCells//(col+1)
    st_h=rowCells//(row+1)
    #print(st_w,st_h)
    ini_w=0
    ini_h=0
    anchors = {}
    count = 0
    for i in range(row):
        ini_h+=st_h
        ini_w=0
        for j in range(col):
            ini_w+=st_w
            anchors[count] = (ini_w,ini_h)
            count+=1
    return anchors

aac = ga(60,90)
aac

In [ ]:
# Train the model each generation and show predictions against the validation
# dataset.

class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'
    
    
for iteration in range(1, 200):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        correct = firstConstraint.decodeConstraint(rowy[0])
        guess = firstConstraint.decodeConstraint(preds[0], calc_argmax=False)
        
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 1
9966/9966 [==============================] - 1896s 190ms/step - loss: 0.6888 - accuracy: 0.7857 - val_loss: 0.6352 - val_accuracy: 0.8021
T [7, 20, 24, 6, 11] ☒ [7, 20, 24, 6, 13]
T [32, 19, 15, 6, 11] ☑ [32, 19, 15, 6, 11]
T [24, 4, 25, 3, 9] ☒ [39, 11, 34, 3, 5]
T [32, 19, 15, 6, 9] ☑ [32, 19, 15, 6, 9]
T [7, 20, 24, 6, 9] ☑ [7, 20, 24, 6, 9]
T [7, 26, 12, 7, 6] ☒ [38, 35, 3, 3, 6]
T [32, 19, 15, 6, 11] ☑ [32, 19, 15, 6, 11]
T [32, 19, 15, 6, 10] ☑ [32, 19, 15, 6, 10]
T [8, 26, 11, 4, 4] ☑ [8, 26, 11, 4, 4]
T [32, 19, 15, 6, 10] ☒ [32, 19, 15, 6, 11]

--------------------------------------------------
Iteration 2
9966/9966 [==============================] - 1894s 190ms/step - loss: 0.5947 - accuracy: 0.8126 - val_loss: 0.5799 - val_accuracy: 0.8166
T [1, 4, 36, 3, 8] ☑ [1, 4, 36, 3, 8]
T [0, 3, 35, 3, 8] ☑ [0, 3, 35, 3, 8]
T [0, 3, 35, 3, 8] ☑ [0, 3, 35, 3, 8]
T [0, 3, 35, 3, 8] ☑ [0, 3, 35, 3, 8]
T [38, 35, 3, 3, 8] ☑ [

T [7, 20, 24, 6, 10] ☑ [7, 20, 24, 6, 10]
T [39, 36, 4, 3, 8] ☑ [39, 36, 4, 3, 8]
T [23, 11, 10, 5, 5] ☑ [23, 11, 10, 5, 5]

--------------------------------------------------
Iteration 13
9966/9966 [==============================] - 1897s 190ms/step - loss: 0.3413 - accuracy: 0.8898 - val_loss: 0.4475 - val_accuracy: 0.8631
T [32, 19, 15, 6, 12] ☑ [32, 19, 15, 6, 12]
T [32, 19, 15, 6, 9] ☑ [32, 19, 15, 6, 9]
T [7, 20, 24, 6, 9] ☒ [15, 26, 32, 8, 9]
T [24, 11, 7, 6, 12] ☒ [24, 11, 7, 6, 11]
T [1, 4, 36, 3, 8] ☑ [1, 4, 36, 3, 8]
T [23, 28, 5, 5, 9] ☑ [23, 28, 5, 5, 9]
T [36, 11, 19, 2, 1] ☑ [36, 11, 19, 2, 1]
T [2, 18, 4, 3, 8] ☑ [2, 18, 4, 3, 8]
T [25, 20, 24, 4, 12] ☒ [32, 4, 24, 4, 8]
T [38, 35, 3, 3, 8] ☑ [38, 35, 3, 3, 8]

--------------------------------------------------
Iteration 14
9966/9966 [==============================] - 1899s 191ms/step - loss: 0.3325 - accuracy: 0.8925 - val_loss: 0.4469 - val_accuracy: 0.8636
T [7, 20, 24, 6, 11] ☑ [7, 20, 24, 6, 11]
T [2, 5, 37, 3, 8] 

T [0, 3, 35, 3, 8] ☑ [0, 3, 35, 3, 8]
T [7, 20, 24, 6, 13] ☑ [7, 20, 24, 6, 13]
T [7, 20, 24, 6, 10] ☑ [7, 20, 24, 6, 10]
T [23, 4, 11, 2, 5] ☒ [24, 35, 11, 4, 5]
T [39, 36, 4, 3, 8] ☑ [39, 36, 4, 3, 8]
T [7, 18, 24, 8, 9] ☒ [39, 18, 24, 8, 9]

--------------------------------------------------
Iteration 25
9966/9966 [==============================] - 1892s 190ms/step - loss: 0.2667 - accuracy: 0.9132 - val_loss: 0.4665 - val_accuracy: 0.8660
T [32, 19, 15, 6, 9] ☒ [32, 19, 37, 6, 8]
T [32, 19, 15, 6, 13] ☑ [32, 19, 15, 6, 13]
T [8, 29, 39, 8, 9] ☑ [8, 29, 39, 8, 9]
T [24, 11, 7, 6, 10] ☒ [24, 11, 7, 6, 12]
T [7, 20, 24, 6, 10] ☑ [7, 20, 24, 6, 10]
T [38, 35, 3, 3, 8] ☑ [38, 35, 3, 3, 8]
T [38, 35, 3, 3, 8] ☑ [38, 35, 3, 3, 8]
T [32, 19, 15, 6, 10] ☑ [32, 19, 15, 6, 10]
T [7, 20, 24, 6, 9] ☑ [7, 20, 24, 6, 9]
T [0, 3, 35, 3, 8] ☑ [0, 3, 35, 3, 8]

--------------------------------------------------
Iteration 26
9966/9966 [==============================] - 1886s 189ms/step - loss: 0.2627

In [132]:
for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        correct = firstConstraint.decodeConstraint(rowy[0])
        guess = firstConstraint.decodeConstraint(preds[0], calc_argmax=False)
        
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)

T [1, 4, 36, 3, 8] ☑ [1, 4, 36, 3, 8]
T [7, 20, 24, 6, 10] ☑ [7, 20, 24, 6, 10]
T [1, 4, 36, 3, 8] ☑ [1, 4, 36, 3, 8]
T [2, 29, 38, 7, 5] ☒ [32, 19, 15, 6, 16]
T [32, 19, 15, 6, 10] ☒ [32, 19, 15, 6, 12]
T [38, 35, 3, 3, 8] ☑ [38, 35, 3, 3, 8]
T [23, 18, 19, 2, 4] ☒ [32, 21, 15, 8, 8]
T [23, 11, 10, 5, 5] ☒ [39, 36, 4, 3, 8]
T [0, 3, 35, 3, 8] ☒ [38, 35, 3, 3, 8]
T [7, 20, 24, 6, 10] ☑ [7, 20, 24, 6, 10]
